# Patch Based networks

The following models utilise a patch based ensemble architecture. As a part of the ablation study, we train a single modal patch based model for the MRI and PET. MRI and PET scans are divided into 27 uniform patches of size 44x45x44. There are two classifcation stages for the final single modal patch based model: 
- Patch Feature Extraction: 
    - 27 ResNet models are trained on patches from each patch location to extract local features.
    3D ResNet architecture code sourced from https://github.com/kenshohara/3D-ResNets-PyTorch
- Patch fusion:
    - Feature maps of the 27 patch models are concatenated and used as inputs to a final model for global classification

# Importing Libraries

In [1]:
# importing the libraries
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
import os
import gc
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.preprocessing import StandardScaler

 
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import *
import nibabel as nb
import torchio as tio

from models import (cnn, C3DNet, resnet, ResNetV2, ResNeXt, ResNeXtV2, WideResNet, PreActResNet,
        EfficientNet, DenseNet, ShuffleNet, ShuffleNetV2, SqueezeNet, MobileNet, MobileNetV2)




## Read in images

In [2]:
# Subject IDs who are progressive normal cognition
PNC = pd.read_pickle('PNC.pkl')

# Subject IDs who are stable normal cognition
SNC = pd.read_pickle('SNC.pkl')

In [3]:
# Create datasets

def read_image_data(input_path):
    X_train = []
    y_train = []
    ids = []
    for filename in sorted(os.listdir(input_path)):
        file = os.path.join(input_path, filename)
        img_file = nb.load(file)
        img = img_file.get_fdata()
        X_train.append(img)
        ids.append(filename)
        # Progressive normal cognition target class 1
        if filename[0:8] in np.array(PNC):
            y_train.append(1)
        # Stable normal cognition class 0
        else:
            y_train.append(0)

    X_train = np.array(X_train)
    y_train = np.array(y_train)

    # Reshape X_train to include channel dimension
    X_train = X_train.reshape(X_train.shape + (1,))
    return X_train, y_train, ids

X_MRI, y_MRI, ids_MRI = read_image_data("E:/Work/Processed_MRI/2.MNI_Registration")
X_PET, y_PET, ids_PET = read_image_data("E:/Work/Processed_PIB/4.MNI_Registered")


## Creating training and test sets for images

In [4]:
# 0.6 - 0.2 - 0.2 train-val-test split
X_train_MRI, X_test_MRI, X_train_PET, X_test_PET, ids_train, ids_test, y_train, y_test = train_test_split(X_MRI, 
                                                                                     X_PET, ids_MRI, 
                                                                                     y_MRI, test_size=0.2, 
                                                                                     random_state=101, stratify=y_MRI)

X_train_MRI, X_val_MRI, X_train_PET, X_val_PET, ids_train, ids_val, y_train, y_val = train_test_split(X_train_MRI, X_train_PET, 
                                                                                      ids_train, y_train, 
                                                                                      test_size=0.25, random_state=101,
                                                                                      stratify=y_train)

In [5]:
# Convert datasets to tensor format, with channel first
train_x_MRI = torch.from_numpy(X_train_MRI).float().permute(0,4,1,2,3)
train_x_PET = torch.from_numpy(X_train_PET).float().permute(0,4,1,2,3)
train_y = torch.from_numpy(y_train).float()

val_x_MRI = torch.from_numpy(X_val_MRI).float().permute(0,4,1,2,3)
val_x_PET = torch.from_numpy(X_val_PET).float().permute(0,4,1,2,3)
val_y = torch.from_numpy(y_val).float()

test_x_MRI = torch.from_numpy(X_test_MRI).float().permute(0,4,1,2,3)
test_x_PET = torch.from_numpy(X_test_PET).float().permute(0,4,1,2,3)
test_y = torch.from_numpy(y_test).float()

## Perform data augmentation to increase training set size

In [ ]:
def perform_augmentation(dataset, seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

    # Define transformations
    training_transform = tio.Compose([
        tio.RandomAffine(),
        tio.RandomElasticDeformation(),
        tio.RandomFlip()
    ])
    augmented_dataset = torch.clone(dataset) 
    for i in range(len(augmented_dataset)):
        augmented_dataset[i] = training_transform(augmented_dataset[i])
    return augmented_dataset

orig_train_x_MRI = torch.clone(train_x_MRI)
orig_train_x_PET = torch.clone(train_x_PET)
orig_train_y = torch.clone(train_y)
for seed in [1,101,42]:
    # Apply transformations and create augmented training set
    augmented_train_MRI = perform_augmentation(orig_train_x_MRI, seed)
    augmented_train_PET = perform_augmentation(orig_train_x_PET, seed)
    
    
    # Concatenate training and augmented training datasets
    train_x_MRI = torch.cat((train_x_MRI, augmented_train_MRI), 0)
    train_x_PET = torch.cat((train_x_PET, augmented_train_PET), 0)
    train_y = torch.cat((train_y, orig_train_y), 0)

# Removing blank space around brain to make dimensions even so we can divide image into uniform patches
train_x_MRI = train_x_MRI[:, :, 0:88, 0:108, 0:88]
train_x_PET = train_x_PET[:, :, 0:88, 0:108, 0:88]   

In [ ]:
# Load augmented dataset (for reruns)
train_x_MRI = torch.load("train_x_MRI.pkl")
train_x_PET = torch.load("train_x_PET.pkl")
train_y = torch.load("train_y.pkl")

## Divide images into patches

In [ ]:
# For each sample, divide images into 27 uniform 3x3x3 patches of size 44x54x44 with 50% overlap
# Then create 27 training datasets for each patch location 
# e.g a data set for patches of each subject in the top left corner, ..., a dataset for patches of each subject in the middle

def create_patches(images, dim, labels):
    # Create datasets for each patch location
    datasets = [[] for _ in range(27)]
    
    # Find the starting locations of each patch
    starting_points = []
    for height_stride in range(3):
        for width_stride in range(3):
            for depth_stride in range(3):
                    start = (0 + height_stride*dim[1]//4, 0 + width_stride*dim[2]//4, 0 + depth_stride*dim[3]//4)
                    starting_points.append(start)

    # For each image
    for i in range(len(images)):
        # Create patch from every starting point
        for j in range(len(starting_points)):
            start_pt = starting_points[j]
            patch = images[i][:, start_pt[0]:start_pt[0] + dim[1]//2, 
                                      start_pt[1]:start_pt[1] + dim[2]//2, 
                                      start_pt[2]:start_pt[2] + dim[3]//2]
            datasets[j].append(patch)
            
    # For each patch location, stack patches from every sample samples into a tensor as a Tensor Dataset
    for i in range(len(datasets)):
        datasets[i] = torch.stack(datasets[i])
        datasets[i] = torch.utils.data.TensorDataset(datasets[i],labels)
    return datasets


In [ ]:
# Create datasets for each patch location

dim = train_x_MRI[0].shape
patch_train_datasets_MRI = create_patches(train_x_MRI, dim, train_y)
patch_train_datasets_PET = create_patches(train_x_PET, dim, train_y)

patch_val_datasets_MRI = create_patches(val_x_MRI, dim, val_y)
patch_val_datasets_PET = create_patches(val_x_PET, dim, val_y)

patch_test_datasets_MRI = create_patches(test_x_MRI, dim, test_y)
patch_test_datasets_PET = create_patches(test_x_PET, dim, test_y)


# Create data loaders for each patch location dataset
patch_train_dataloaders_MRI = []
patch_train_dataloaders_PET = []

patch_val_dataloaders_MRI = []
patch_val_dataloaders_PET = []

patch_test_dataloaders_MRI = []
patch_test_dataloaders_PET = []

batch_size = 16
for i in range(27):
    patch_train_dataloaders_MRI.append(torch.utils.data.DataLoader(patch_train_datasets_MRI[i], 
                                                                  batch_size = batch_size, shuffle = True))
    patch_train_dataloaders_PET.append(torch.utils.data.DataLoader(patch_train_datasets_PET[i], 
                                                                  batch_size = batch_size, shuffle = True))
    
    patch_val_dataloaders_MRI.append(torch.utils.data.DataLoader(patch_val_datasets_MRI[i], 
                                                                  batch_size = batch_size, shuffle = False))
    patch_val_dataloaders_PET.append(torch.utils.data.DataLoader(patch_val_datasets_PET[i], 
                                                                  batch_size = batch_size, shuffle = False))
    
    patch_test_dataloaders_MRI.append(torch.utils.data.DataLoader(patch_test_datasets_MRI[i], 
                                                                  batch_size = batch_size, shuffle = False))
    patch_test_dataloaders_PET.append(torch.utils.data.DataLoader(patch_test_datasets_PET[i], 
                                                                  batch_size = batch_size, shuffle = False))

In [ ]:
# Define device being used for model training
if torch.cuda.is_available(): 
    dev = "cuda:0" 
else: 
    dev = "cpu" 
device = torch.device(dev) 

# Model training

In [ ]:
def train_model(model, train_loader, optimiser, error, scheduler, feature_map=False):
    total_train_loss = 0
    for image, labels in train_loader:
        image = image.to(device)
        labels = labels.to(device)
        
        # Clear gradients
        optimiser.zero_grad()
        
        # Forward propagation
        # feature_map indicates whether the feature map prior final layer is also included in the outputs of the model
        if feature_map:
            outputs = model(image)[1]
        else:
            outputs = model(image)
        
        # Calculate loss
        loss = error(outputs.flatten(), labels)
        
        total_train_loss += loss.item()
        
        # Calculating gradients
        loss.backward()

        # Update parameters
        optimiser.step()
        scheduler.step()
        
        # Clear GPU Cache
        torch.cuda.empty_cache()
        gc.collect()
        
        # Calculate accuracy
        preds = outputs.flatten().round()
    print("Average Training Loss", total_train_loss/len(train_loader.dataset))

def validate_model(model, val_loader, error, feature_map=False):
    correct_predictions_val = 0
    total_val_loss = 0
    with torch.no_grad():
        for image, labels in val_loader:
            image = image.to(device)
            labels = labels.to(device)

            # Forward propagation
            if feature_map:
                pred = model(image)[1]
            else:
                pred = model(image)
            
            # Calculate loss
            loss = error(pred.flatten(), labels)
            total_val_loss += loss.item()

            # Clear GPU Cache
            torch.cuda.empty_cache()
            gc.collect()

            # Calculate accuracy
            preds = pred.flatten().round()
            #print(pred)
            correct_predictions_val += torch.sum(preds == labels).item()
        print("Validation Accuracy:", correct_predictions_val/len(val_loader.dataset))
        print("Average Validation Loss:", total_val_loss/len(val_loader.dataset))
    return total_val_loss/len(val_loader.dataset)
        
def evaluate_model(model, test_loader, error, feature_map=False):
    correct_predictions_test = 0
    preds = []
    labels = []
    with torch.no_grad():
        for test,label in test_loader:
            test = test.to(device)
            label = label.to(device)
            labels.append(label.cpu())
            
            # Forward propagation
            if feature_map:
                pred = model(test)[1]
            else:
                pred = model(test)

            # Clear GPU Cache
            torch.cuda.empty_cache()
            gc.collect()

            # Calculate accuracy
            pred = pred.flatten().round()
            preds.append(pred.cpu())
            correct_predictions_test += torch.sum(pred == label).item()
    print("Test Accuracy:", correct_predictions_test/len(test_loader.dataset))    
    print("Recall:", recall_score(torch.cat(labels), torch.cat(preds)))
    print("Precision:", precision_score(torch.cat(labels), torch.cat(preds)))
    print(preds)
    print(labels)

## Training MRI individual patch models

In [11]:
# Set seeds 
torch.manual_seed(101)
torch.cuda.manual_seed(101)
torch.cuda.manual_seed_all(101)
random.seed(101)
np.random.seed(101)
torch.backends.cudnn.benchmark = False

mri_models = []
# For each patch location, train a ResNet model on the image patches in that location
for i in range(27):
    patch_model = ResNetV2.generate_model(
        model_depth=10,
        n_classes=1,
        n_input_channels=1,
        shortcut_type='B',
        conv1_t_size=7,
        conv1_t_stride=2,
        no_max_pool=False,
        widen_factor=1.0).to(device)
    
    best_val_loss = np.inf
    patience = 15
    no_improvement = 0
    
    # Binary Cross Entropy Loss
    error = nn.BCELoss()

    # SGD Optimizer
    optimiser = SGD(patch_model.parameters(), lr=0.001, momentum=0.9)
    scheduler = torch.optim.lr_scheduler.LinearLR(optimiser, start_factor=1.0, end_factor=0.1, total_iters=10)
    
    for epoch in range(500):
        print(f"----------------------------EPOCH {epoch} PATCH {i}-------------------------------")
        patch_model.train()
        train_model(patch_model, patch_train_dataloaders_MRI[i], optimiser, error, scheduler, True)

        patch_model.eval()
        val_loss = validate_model(patch_model, patch_val_dataloaders_MRI[i], error, True)
        
        # Save model weights if improvement seen
        # Otherwise stop model training if there is no improvement in loss after "patience" number of runs
        if val_loss <= best_val_loss:
            best_val_loss = val_loss
            no_improvement = 0
            torch.save(patch_model.state_dict(), f"./patch_temp/PATCH_TEMP{i}")
        else:
            no_improvement += 1
            if no_improvement <= patience:
                continue
            else:
                print(f"BEST VAL ACC: {best_val_loss}")
                break
         
    print("----------------------------TEST RESULTS-------------------------------")
    patch_model.load_state_dict(torch.load(f"./patch_temp/PATCH_TEMP{i}"))
    patch_model.eval()
    evaluate_model(patch_model, patch_test_dataloaders_MRI[i], error, True)
    mri_models.append(patch_model)

----------------------------EPOCH 0 PATCH 0-------------------------------
Average Training Loss 0.045261246014813905
Validation Accuracy: 0.5121951219512195
Average Validation Loss: 0.051218640513536404
----------------------------EPOCH 1 PATCH 0-------------------------------
Average Training Loss 0.04433265340621354
Validation Accuracy: 0.5121951219512195
Average Validation Loss: 0.050721578481720715
----------------------------EPOCH 2 PATCH 0-------------------------------
Average Training Loss 0.04368459506601584
Validation Accuracy: 0.5365853658536586
Average Validation Loss: 0.05029208049541566
----------------------------EPOCH 3 PATCH 0-------------------------------
Average Training Loss 0.0430848239386668
Validation Accuracy: 0.5853658536585366
Average Validation Loss: 0.05044107320832043
----------------------------EPOCH 4 PATCH 0-------------------------------
Average Training Loss 0.042889644254426484
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.05027

Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.04956503612239186
----------------------------EPOCH 11 PATCH 1-------------------------------
Average Training Loss 0.040049834818136495
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.04919209567511954
----------------------------EPOCH 12 PATCH 1-------------------------------
Average Training Loss 0.03964536734780327
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.04926833873841821
----------------------------EPOCH 13 PATCH 1-------------------------------
Average Training Loss 0.03955713842735916
Validation Accuracy: 0.5853658536585366
Average Validation Loss: 0.04907287766293782
----------------------------EPOCH 14 PATCH 1-------------------------------
Average Training Loss 0.03896153168600114
Validation Accuracy: 0.5609756097560976
Average Validation Loss: 0.04938082433328396
----------------------------EPOCH 15 PATCH 1-------------------------------
Average Training Loss 0.03

Average Training Loss 0.04297787818263789
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.05044393568504148
----------------------------EPOCH 4 PATCH 2-------------------------------
Average Training Loss 0.042648761365257325
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.05039556869646398
----------------------------EPOCH 5 PATCH 2-------------------------------
Average Training Loss 0.04248216723809477
Validation Accuracy: 0.5365853658536586
Average Validation Loss: 0.05054363971803247
----------------------------EPOCH 6 PATCH 2-------------------------------
Average Training Loss 0.04209931023785325
Validation Accuracy: 0.5365853658536586
Average Validation Loss: 0.05039114050748872
----------------------------EPOCH 7 PATCH 2-------------------------------
Average Training Loss 0.04185357677643416
Validation Accuracy: 0.5853658536585366
Average Validation Loss: 0.05066252917778201
----------------------------EPOCH 8 PATCH 2---------------------

Average Training Loss 0.04044466563424126
Validation Accuracy: 0.6585365853658537
Average Validation Loss: 0.050012485283177074
----------------------------EPOCH 15 PATCH 3-------------------------------
Average Training Loss 0.039991287422961874
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.049844390008507704
----------------------------EPOCH 16 PATCH 3-------------------------------
Average Training Loss 0.039708612639395915
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.05007364546380392
----------------------------EPOCH 17 PATCH 3-------------------------------
Average Training Loss 0.039288117748792054
Validation Accuracy: 0.6585365853658537
Average Validation Loss: 0.04999038068259635
----------------------------EPOCH 18 PATCH 3-------------------------------
Average Training Loss 0.039178080734659414
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.049960249807776476
----------------------------EPOCH 19 PATCH 3----------

Average Training Loss 0.0405096309106858
Validation Accuracy: 0.5609756097560976
Average Validation Loss: 0.049986302852630615
----------------------------EPOCH 10 PATCH 4-------------------------------
Average Training Loss 0.040108483956485495
Validation Accuracy: 0.5609756097560976
Average Validation Loss: 0.050232993393409545
----------------------------EPOCH 11 PATCH 4-------------------------------
Average Training Loss 0.0399247372492415
Validation Accuracy: 0.5853658536585366
Average Validation Loss: 0.05005802323178547
----------------------------EPOCH 12 PATCH 4-------------------------------
Average Training Loss 0.03928011264957366
Validation Accuracy: 0.5121951219512195
Average Validation Loss: 0.050110353202354616
----------------------------EPOCH 13 PATCH 4-------------------------------
Average Training Loss 0.03904655807819523
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.05041370740750941
----------------------------EPOCH 14 PATCH 4---------------

...

Average Training Loss 0.0345385947921237
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.04919816953379933
----------------------------EPOCH 27 PATCH 21-------------------------------
Average Training Loss 0.034228430297531066
Validation Accuracy: 0.6585365853658537
Average Validation Loss: 0.049280152088258325
----------------------------EPOCH 28 PATCH 21-------------------------------
Average Training Loss 0.033547718138968355
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.04952181984738606
----------------------------EPOCH 29 PATCH 21-------------------------------
Average Training Loss 0.03256280842374583
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.04923831689648512
----------------------------EPOCH 30 PATCH 21-------------------------------
Average Training Loss 0.032248389097999354
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.05000117639215981
----------------------------EPOCH 31 PATCH 21---------

Average Training Loss 0.033349681523491125
Validation Accuracy: 0.5853658536585366
Average Validation Loss: 0.050411750630634584
----------------------------EPOCH 27 PATCH 22-------------------------------
Average Training Loss 0.03168268764361006
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.0524211147936379
----------------------------EPOCH 28 PATCH 22-------------------------------
Average Training Loss 0.031017482097520203
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.05085037539644939
----------------------------EPOCH 29 PATCH 22-------------------------------
Average Training Loss 0.031059009435235478
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.05215768407030803
----------------------------EPOCH 30 PATCH 22-------------------------------
Average Training Loss 0.030710368371400676
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.05072801578335646
----------------------------EPOCH 31 PATCH 22--------

Average Training Loss 0.04185788186847186
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.04966881217026129
----------------------------EPOCH 8 PATCH 24-------------------------------
Average Training Loss 0.0415856843111945
Validation Accuracy: 0.7317073170731707
Average Validation Loss: 0.049587649543110915
----------------------------EPOCH 9 PATCH 24-------------------------------
Average Training Loss 0.041352257621092875
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.04945318291826946
----------------------------EPOCH 10 PATCH 24-------------------------------
Average Training Loss 0.041049264004973114
Validation Accuracy: 0.7073170731707317
Average Validation Loss: 0.04936685503982916
----------------------------EPOCH 11 PATCH 24-------------------------------
Average Training Loss 0.040796665871729615
Validation Accuracy: 0.7073170731707317
Average Validation Loss: 0.0494519937329176
----------------------------EPOCH 12 PATCH 24------------

Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.048243417972471656
----------------------------EPOCH 48 PATCH 24-------------------------------
Average Training Loss 0.020810007407772738
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.049456020680869496
----------------------------EPOCH 49 PATCH 24-------------------------------
Average Training Loss 0.020393582946452937
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.050589718469759314
----------------------------EPOCH 50 PATCH 24-------------------------------
Average Training Loss 0.018956224510415655
Validation Accuracy: 0.5853658536585366
Average Validation Loss: 0.05163601840414652
----------------------------EPOCH 51 PATCH 24-------------------------------
Average Training Loss 0.01984237146670701
Validation Accuracy: 0.6585365853658537
Average Validation Loss: 0.05029071249612948
----------------------------EPOCH 52 PATCH 24-------------------------------
Average Training

Average Training Loss 0.03377650385020209
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.050814954245962746
----------------------------EPOCH 28 PATCH 25-------------------------------
Average Training Loss 0.03347584837284245
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.050703759600476524
----------------------------EPOCH 29 PATCH 25-------------------------------
Average Training Loss 0.0332380158857244
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.05227112188571837
BEST VAL ACC: 0.05026206301479805
----------------------------TEST RESULTS-------------------------------
Test Accuracy: 0.5853658536585366
Recall: 0.7619047619047619
Precision: 0.5714285714285714
[tensor([1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1.]), tensor([1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0.]), tensor([0., 1., 1., 1., 1., 1., 1., 1., 0.])]
[tensor([1., 0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0.]

In [11]:
# Reload weights (for rerunning)
mri_models = []
for i in range(27):
    patch_model = ResNetV2.generate_model(
        model_depth=10,
        n_classes=1,
        n_input_channels=1,
        shortcut_type='B',
        conv1_t_size=7,
        conv1_t_stride=2,
        no_max_pool=False,
        widen_factor=1.0).to(device)
    patch_model.load_state_dict(torch.load(f"./patch_temp/PATCH_TEMP{i}"))
    mri_models.append(patch_model)
    
    

## Training PET Individual Patch Models

In [12]:
# Set seeds
torch.manual_seed(101)
torch.cuda.manual_seed(101)
random.seed(101)
np.random.seed(101)


pet_models = []
# For each patch location, train a resnet model on the image patches in that location
for i in range(27):
    patch_model = ResNetV2.generate_model(
        model_depth=10,
        n_classes=1,
        n_input_channels=1,
        shortcut_type='B',
        conv1_t_size=7,
        conv1_t_stride=2,
        no_max_pool=False,
        widen_factor=1.0).to(device)
    
    
    best_val_loss = np.inf
    patience = 15
    no_improvement = 0
    
    # Binary Cross Entropy Loss
    error = nn.BCELoss()

    # SGD Optimizer
    optimiser = SGD(patch_model.parameters(), lr=0.001, momentum=0.9)
    scheduler = torch.optim.lr_scheduler.LinearLR(optimiser, start_factor=1.0, end_factor=0.1, total_iters=10)
    
    for epoch in range(500):
        print(f"----------------------------EPOCH {epoch} PATCH {i}-------------------------------")
        patch_model.train()
        train_model(patch_model, patch_train_dataloaders_PET[i], optimiser, error, scheduler, True)

        patch_model.eval()
        val_loss = validate_model(patch_model, patch_val_dataloaders_PET[i], error, True)
        
        # Save model weights if improvement seen
        # Otherwise stop model training if there is no improvement in loss after "patience" number of runs
        if val_loss <= best_val_loss:
            best_val_loss = val_loss
            no_improvement = 0
            torch.save(patch_model.state_dict(), f"./patch_temp/PET_PATCH_TEMP{i}")
        else:
            no_improvement += 1
            if no_improvement <= patience:
                continue
            else:
                print(f"BEST VAL ACC: {best_val_loss}")
                break
         
    print("----------------------------TEST RESULTS-------------------------------")
    patch_model.load_state_dict(torch.load(f"./patch_temp/PET_PATCH_TEMP{i}"))
    patch_model.eval()
    evaluate_model(patch_model, patch_test_dataloaders_PET[i], error, True)
    pet_models.append(patch_model)

----------------------------EPOCH 0 PATCH 0-------------------------------
Average Training Loss 0.04476547876342398
Validation Accuracy: 0.5121951219512195
Average Validation Loss: 0.05084104654265613
----------------------------EPOCH 1 PATCH 0-------------------------------
Average Training Loss 0.04387498390479166
Validation Accuracy: 0.5121951219512195
Average Validation Loss: 0.050569458705622974
----------------------------EPOCH 2 PATCH 0-------------------------------
Average Training Loss 0.043307030909374114
Validation Accuracy: 0.5121951219512195
Average Validation Loss: 0.050627885795221095
----------------------------EPOCH 3 PATCH 0-------------------------------
Average Training Loss 0.04266059203226058
Validation Accuracy: 0.5121951219512195
Average Validation Loss: 0.0502380016373425
----------------------------EPOCH 4 PATCH 0-------------------------------
Average Training Loss 0.042144490070030335
Validation Accuracy: 0.5121951219512195
Average Validation Loss: 0.04996

Average Training Loss 0.02664358541369438
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.047174530785258226
----------------------------EPOCH 41 PATCH 0-------------------------------
Average Training Loss 0.026226759079049845
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.04747197250040566
----------------------------EPOCH 42 PATCH 0-------------------------------
Average Training Loss 0.02530120136063607
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.046454230459724986
----------------------------EPOCH 43 PATCH 0-------------------------------
Average Training Loss 0.025583790035032836
Validation Accuracy: 0.5853658536585366
Average Validation Loss: 0.0478093914869355
----------------------------EPOCH 44 PATCH 0-------------------------------
Average Training Loss 0.02434821561223171
Validation Accuracy: 0.7317073170731707
Average Validation Loss: 0.04737097606426332
----------------------------EPOCH 45 PATCH 0--------------

Validation Accuracy: 0.7317073170731707
Average Validation Loss: 0.05225165151968235
----------------------------EPOCH 81 PATCH 0-------------------------------
Average Training Loss 0.0036511697775882777
Validation Accuracy: 0.7073170731707317
Average Validation Loss: 0.049858613712031666
----------------------------EPOCH 82 PATCH 0-------------------------------
Average Training Loss 0.004205216439898874
Validation Accuracy: 0.6585365853658537
Average Validation Loss: 0.0489451231026068
----------------------------EPOCH 83 PATCH 0-------------------------------
Average Training Loss 0.003167014794240965
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.04897523798593661
BEST VAL ACC: 0.044628695016954004
----------------------------TEST RESULTS-------------------------------
Test Accuracy: 0.7317073170731707
Recall: 0.7142857142857143
Precision: 0.75
[tensor([1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1.]), tensor([0., 0., 1., 1., 0., 1., 0., 1., 1.,

Average Training Loss 0.029903619504365764
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.04786226080685127
----------------------------EPOCH 35 PATCH 1-------------------------------
Average Training Loss 0.029885123071611903
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.04819764305905598
----------------------------EPOCH 36 PATCH 1-------------------------------
Average Training Loss 0.028921860346540075
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.048436606802591466
----------------------------EPOCH 37 PATCH 1-------------------------------
Average Training Loss 0.028930694475525716
Validation Accuracy: 0.6585365853658537
Average Validation Loss: 0.04854791920359542
----------------------------EPOCH 38 PATCH 1-------------------------------
Average Training Loss 0.028566137139425903
Validation Accuracy: 0.7073170731707317
Average Validation Loss: 0.04826600522529788
----------------------------EPOCH 39 PATCH 1-----------

Average Training Loss 0.03928152838202774
Validation Accuracy: 0.5853658536585366
Average Validation Loss: 0.04913986456103441
----------------------------EPOCH 14 PATCH 2-------------------------------
Average Training Loss 0.038891495129124064
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.048960046070378
----------------------------EPOCH 15 PATCH 2-------------------------------
Average Training Loss 0.038707187429803315
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.04896063048665116
----------------------------EPOCH 16 PATCH 2-------------------------------
Average Training Loss 0.0384419213552944
Validation Accuracy: 0.5853658536585366
Average Validation Loss: 0.04911831820883402
----------------------------EPOCH 17 PATCH 2-------------------------------
Average Training Loss 0.038114212209083995
Validation Accuracy: 0.5853658536585366
Average Validation Loss: 0.04889317838157096
----------------------------EPOCH 18 PATCH 2-----------------

...

Average Training Loss 0.04298677466443328
Validation Accuracy: 0.5365853658536586
Average Validation Loss: 0.0508417225465542
----------------------------EPOCH 2 PATCH 24-------------------------------
Average Training Loss 0.042457387584154724
Validation Accuracy: 0.5853658536585366
Average Validation Loss: 0.04991724432968512
----------------------------EPOCH 3 PATCH 24-------------------------------
Average Training Loss 0.04190143344343686
Validation Accuracy: 0.6585365853658537
Average Validation Loss: 0.04968363192023301
----------------------------EPOCH 4 PATCH 24-------------------------------
Average Training Loss 0.04159955106309203
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.049910535172718325
----------------------------EPOCH 5 PATCH 24-------------------------------
Average Training Loss 0.04104026541358135
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.049351075800453746
----------------------------EPOCH 6 PATCH 24---------------

Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.04722302134444074
----------------------------EPOCH 42 PATCH 24-------------------------------
Average Training Loss 0.025382007548554998
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.047586603862483326
----------------------------EPOCH 43 PATCH 24-------------------------------
Average Training Loss 0.024835007662167313
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.04786407366031554
----------------------------EPOCH 44 PATCH 24-------------------------------
Average Training Loss 0.02333724364394047
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.04802975276621377
----------------------------EPOCH 45 PATCH 24-------------------------------
Average Training Loss 0.023283073464866546
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.04736800164711184
----------------------------EPOCH 46 PATCH 24-------------------------------
Average Training L

Average Training Loss 0.03070859537749994
Validation Accuracy: 0.6585365853658537
Average Validation Loss: 0.04503521105138267
----------------------------EPOCH 32 PATCH 25-------------------------------
Average Training Loss 0.030903594537836608
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.04450897472660716
----------------------------EPOCH 33 PATCH 25-------------------------------
Average Training Loss 0.029936564445007044
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.04455022259456355
----------------------------EPOCH 34 PATCH 25-------------------------------
Average Training Loss 0.02952336397815923
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.044364897216238625
----------------------------EPOCH 35 PATCH 25-------------------------------
Average Training Loss 0.02920699137888971
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.04454510822528746
----------------------------EPOCH 36 PATCH 25---------

Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.04729187488555908
----------------------------EPOCH 13 PATCH 26-------------------------------
Average Training Loss 0.03818144863013361
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.04672894972126659
----------------------------EPOCH 14 PATCH 26-------------------------------
Average Training Loss 0.037453236272100544
Validation Accuracy: 0.6585365853658537
Average Validation Loss: 0.04649522246384039
----------------------------EPOCH 15 PATCH 26-------------------------------
Average Training Loss 0.03703836937908266
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.04617563544250116
----------------------------EPOCH 16 PATCH 26-------------------------------
Average Training Loss 0.03686015497221321
Validation Accuracy: 0.6585365853658537
Average Validation Loss: 0.04636480459352819
----------------------------EPOCH 17 PATCH 26-------------------------------
Average Training Loss

Average Training Loss 0.021597171813005307
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.043921746858736364
----------------------------EPOCH 54 PATCH 26-------------------------------
Average Training Loss 0.020290167788501647
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.04351677400309865
----------------------------EPOCH 55 PATCH 26-------------------------------
Average Training Loss 0.020047652672548762
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.04450723601550591
----------------------------EPOCH 56 PATCH 26-------------------------------
Average Training Loss 0.01866880031760599
Validation Accuracy: 0.6585365853658537
Average Validation Loss: 0.045663814719130356
----------------------------EPOCH 57 PATCH 26-------------------------------
Average Training Loss 0.018050572911246877
Validation Accuracy: 0.7073170731707317
Average Validation Loss: 0.0448555481143114
BEST VAL ACC: 0.04330193124166349
------------------

In [12]:
# Reload weights (for rerunning)
pet_models = []
for i in range(27):
    patch_model = ResNetV2.generate_model(
        model_depth=10,
        n_classes=1,
        n_input_channels=1,
        shortcut_type='B',
        conv1_t_size=7,
        conv1_t_stride=2,
        no_max_pool=False,
        widen_factor=1.0).to(device)
    patch_model.load_state_dict(torch.load(f"./patch_temp/PET_PATCH_TEMP{i}"))
    pet_models.append(patch_model)
    

# Training ensemble models integrating the individual patch based models

The models trained below are at the whole image level. Features from the 27 models trained on each patch location will be concatenated and used for the final global classification.

In [13]:
# Stacking all 27 patch based datasets to create a single subject level dataset
# We will now have a subject level dataset where each row is a subject and their 27 patch images
# e.g row 1 would be: [patch_1,....,patch_27] for subject 1

all_patches_train_MRI = torch.stack([dataset.tensors[0] for dataset in patch_train_datasets_MRI], dim=1)
all_patches_val_MRI = torch.stack([dataset.tensors[0] for dataset in patch_val_datasets_MRI], dim=1)
all_patches_test_MRI = torch.stack([dataset.tensors[0] for dataset in patch_test_datasets_MRI], dim=1)

all_patches_train_PET = torch.stack([dataset.tensors[0] for dataset in patch_train_datasets_PET], dim=1)
all_patches_val_PET = torch.stack([dataset.tensors[0] for dataset in patch_val_datasets_PET], dim=1)
all_patches_test_PET = torch.stack([dataset.tensors[0] for dataset in patch_test_datasets_PET], dim=1)

In [14]:
batch_size = 16
all_patches_train_dataloader_MRI = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(all_patches_train_MRI, train_y), 
                                                                  batch_size = batch_size, shuffle = True)
all_patches_val_dataloader_MRI = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(all_patches_val_MRI, val_y), 
                                                                 batch_size = batch_size, shuffle = False)
all_patches_test_dataloader_MRI = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(all_patches_test_MRI, test_y), 
                                                                  batch_size = batch_size, shuffle = False)

all_patches_train_dataloader_PET = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(all_patches_train_PET, train_y), 
                                                                  batch_size = batch_size, shuffle = True)
all_patches_val_dataloader_PET = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(all_patches_val_PET, val_y), 
                                                                  batch_size = batch_size, shuffle = False)
all_patches_test_dataloader_PET = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(all_patches_test_PET, test_y), 
                                                                  batch_size = batch_size, shuffle = False)


In [15]:
# Define our final multi stream network
# This network uses the feature maps of the 27 patch models trained previously above as inputs
class multistream_CNN(nn.Module):
    def __init__(self, patch_models):
        super().__init__()
        self.patch_models = nn.ModuleList(patch_models)
        self.drop= nn.Dropout(p=0.4)
        
        self.fc1 = nn.Linear(2700, 1000) 
        self.rel1 = nn.ReLU()
        self.batch_norm1 = nn.BatchNorm1d(1000)
        
        self.fc2 = nn.Linear(1000, 200) 
        self.rel2 = nn.ReLU()
        self.batch_norm2 = nn.BatchNorm1d(200)
        
        self.fc3 = nn.Linear(2700, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        patch_outputs = []
        for i in range(27):
            patch_output, _ = self.patch_models[i](x[:, i])  
            patch_outputs.append(self.drop(patch_output))
        x = torch.cat(patch_outputs, dim=1)
        x = self.sigmoid(self.fc3(x))
        return x

### MRI ensemble patch model

In [16]:
MRI_patch_model = multistream_CNN(mri_models).to(device)

#Freeze layers of the 27 individual patch models
for patch_model in MRI_patch_model.patch_models:
    for name, param in patch_model.named_parameters():      
            param.requires_grad = False 

# CNN model training
torch.manual_seed(101)
torch.cuda.manual_seed(101)
random.seed(101)
np.random.seed(101)

# Binary Cross Entropy Loss
error = nn.BCELoss()

# SGD Optimizer
optimiser = SGD(MRI_patch_model.parameters(), lr=0.0001, momentum=0.9)
scheduler = lr_scheduler.LinearLR(optimiser, start_factor=1.0, end_factor=0.1, total_iters=10)

# Validation Hyperparameters for early stopping
best_val_loss = np.inf
patience = 20
no_improvement = 0

for epoch in range(500):
    print(f"----------------------------EPOCH {epoch}-------------------------------")
    MRI_patch_model.train()
    train_model(MRI_patch_model, all_patches_train_dataloader_MRI, optimiser, error, scheduler)
    
    MRI_patch_model.eval()
    avg_val_loss = validate_model(MRI_patch_model, all_patches_val_dataloader_MRI, error)
    
    # Save model weights if improvement seen
    # Otherwise stop model training if there is no improvement in loss after "patience" number of runs
    if avg_val_loss <= best_val_loss:
        best_val_loss = avg_val_loss
        no_improvement = 0
        torch.save(MRI_patch_model.state_dict(), "./trained_models_2/MRI_PATCH_MODEL")
    else:
        no_improvement += 1
        if no_improvement <= patience:
            continue
        else:
            print(f"BEST VAL LOSS: {best_val_loss}")
            break
            
print("----------------------------TEST RESULTS-------------------------------")
MRI_patch_model.load_state_dict(torch.load("./trained_models_2/MRI_PATCH_MODEL"))
MRI_patch_model.eval()
evaluate_model(MRI_patch_model, all_patches_test_dataloader_MRI, error)

----------------------------EPOCH 0-------------------------------
Average Training Loss 0.04318771047181771
Validation Accuracy: 0.5365853658536586
Average Validation Loss: 0.050292099394449376
----------------------------EPOCH 1-------------------------------
Average Training Loss 0.042315416893021006
Validation Accuracy: 0.5853658536585366
Average Validation Loss: 0.05009123610287178
----------------------------EPOCH 2-------------------------------
Average Training Loss 0.04201488367846755
Validation Accuracy: 0.6585365853658537
Average Validation Loss: 0.049873013321946304
----------------------------EPOCH 3-------------------------------
Average Training Loss 0.04161110068442392
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.04972493357774688
----------------------------EPOCH 4-------------------------------
Average Training Loss 0.04103362035067355
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.049557411089176086
--------------------------

Average Training Loss 0.029443663651825953
Validation Accuracy: 0.5853658536585366
Average Validation Loss: 0.048059831305247984
----------------------------EPOCH 43-------------------------------
Average Training Loss 0.028512269258499146
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.04809697662911764
----------------------------EPOCH 44-------------------------------
Average Training Loss 0.02867675896306507
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.04810806454681769
----------------------------EPOCH 45-------------------------------
Average Training Loss 0.029172322850246898
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.04816103563076112
----------------------------EPOCH 46-------------------------------
Average Training Loss 0.028778458472158087
Validation Accuracy: 0.5853658536585366
Average Validation Loss: 0.0482183782065787
----------------------------EPOCH 47-------------------------------
Average Training Loss

In [6]:
# Print which subjects the MRI patch model classified correctly
y = [torch.tensor([1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 1.]),
     torch.tensor([1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0.]),
     torch.tensor([0., 1., 0., 1., 1., 1., 1., 1., 0.])]


predictions = [torch.tensor([1., 0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0.]),
               torch.tensor([0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0.]),
               torch.tensor([0., 0., 0., 1., 1., 1., 1., 1., 0.])]
y = torch.cat(y)
predictions = torch.cat(predictions)
correct = y==predictions
for i in range(41):
    if correct[i]:
        print(ids_test[i])

OAS30314_MR_d.nii.gz
OAS30146_MR_d.nii.gz
OAS30895_MR_d.nii.gz
OAS30830_MR_d.nii.gz
OAS31059_MR_d.nii.gz
OAS30079_MR_d.nii.gz
OAS30986_MR_d.nii.gz
OAS30929_MR_d.nii.gz
OAS31125_MR_d.nii.gz
OAS30241_MR_d.nii.gz
OAS30032_MR_d.nii.gz
OAS30224_MR_d.nii.gz
OAS30632_MR_d.nii.gz
OAS30921_MR_d.nii.gz
OAS30614_MR_d.nii.gz
OAS31092_MR_d.nii.gz
OAS31162_MR_d.nii.gz
OAS30041_MR_d.nii.gz
OAS31168_MR_d.nii.gz
OAS30028_MR_d.nii.gz
OAS30015_MR_d.nii.gz
OAS30735_MR_d.nii.gz
OAS31103_MR_d.nii.gz
OAS30749_MR_d.nii.gz
OAS30991_MR_d.nii.gz
OAS30223_MR_d.nii.gz
OAS30964_MR_d.nii.gz
OAS30572_MR_d.nii.gz
OAS31087_MR_d.nii.gz


### PET ensemble patch model

In [17]:
PET_patch_model = multistream_CNN(pet_models).to(device)

#Freeze layers of individual patch models
for patch_model in PET_patch_model.patch_models:
    for name, param in patch_model.named_parameters():
        param.requires_grad = False 

# CNN model training
torch.manual_seed(101)
torch.cuda.manual_seed(101)
random.seed(101)
np.random.seed(101)

# Binary Cross Entropy Loss
error = nn.BCELoss()

# SGD Optimizer
optimiser = SGD(PET_patch_model.parameters(), lr=0.0001, momentum=0.9)
scheduler = lr_scheduler.LinearLR(optimiser, start_factor=1.0, end_factor=0.1, total_iters=10)

# Validation Hyperparameters for early stopping
best_val_loss = np.inf
patience = 20
no_improvement = 0

for epoch in range(1000):
    print(f"----------------------------EPOCH {epoch}-------------------------------")
    PET_patch_model.train()
    train_model(PET_patch_model, all_patches_train_dataloader_PET, optimiser, error, scheduler)
    
    PET_patch_model.eval()
    avg_val_loss = validate_model(PET_patch_model, all_patches_val_dataloader_PET, error)
    
    # Save model weights if improvement seen
    # Otherwise stop model training if there is no improvement in loss after "patience" number of runs
    if avg_val_loss <= best_val_loss:
        best_val_loss = avg_val_loss
        no_improvement = 0
        torch.save(PET_patch_model.state_dict(), "./trained_models_2/PET_PATCH_MODEL")
    else:
        no_improvement += 1
        if no_improvement <= patience:
            continue
        else:
            print(f"BEST VAL LOSS: {best_val_loss}")
            break
            
print("----------------------------TEST RESULTS-------------------------------")
PET_patch_model.load_state_dict(torch.load("./trained_models_2/PET_PATCH_MODEL"))
PET_patch_model.eval()
evaluate_model(PET_patch_model, all_patches_test_dataloader_PET, error)

----------------------------EPOCH 0-------------------------------
Average Training Loss 0.04209426961472777
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.050074503189179956
----------------------------EPOCH 1-------------------------------
Average Training Loss 0.039634323266685985
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.04913099655290929
----------------------------EPOCH 2-------------------------------
Average Training Loss 0.03778465861668352
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.04830974049684478
----------------------------EPOCH 3-------------------------------
Average Training Loss 0.03555525020986307
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.047636950888284825
----------------------------EPOCH 4-------------------------------
Average Training Loss 0.03402924977365087
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.04717466598603784
---------------------------

In [7]:
# Print which subjects the PET patch model classified correctly
y = [torch.tensor([1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1.]),
     torch.tensor([0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0.]),
     torch.tensor([0., 0., 0., 1., 1., 1., 1., 1., 0.])]


predictions = [torch.tensor([1., 0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0.]),
               torch.tensor([0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0.]),
               torch.tensor([0., 0., 0., 1., 1., 1., 1., 1., 0.])]
y = torch.cat(y)
predictions = torch.cat(predictions)
correct = y==predictions
for i in range(41):
    if correct[i]:
        print(ids_test[i])

OAS30314_MR_d.nii.gz
OAS30146_MR_d.nii.gz
OAS30895_MR_d.nii.gz
OAS30830_MR_d.nii.gz
OAS31059_MR_d.nii.gz
OAS30079_MR_d.nii.gz
OAS30986_MR_d.nii.gz
OAS30929_MR_d.nii.gz
OAS31125_MR_d.nii.gz
OAS30032_MR_d.nii.gz
OAS30907_MR_d.nii.gz
OAS30921_MR_d.nii.gz
OAS30755_MR_d.nii.gz
OAS31344_MR_d.nii.gz
OAS30101_MR_d.nii.gz
OAS30769_MR_d.nii.gz
OAS30933_MR_d.nii.gz
OAS30959_MR_d.nii.gz
OAS31168_MR_d.nii.gz
OAS30028_MR_d.nii.gz
OAS30015_MR_d.nii.gz
OAS30735_MR_d.nii.gz
OAS30160_MR_d.nii.gz
OAS31103_MR_d.nii.gz
OAS30749_MR_d.nii.gz
OAS30991_MR_d.nii.gz
OAS30223_MR_d.nii.gz
OAS30964_MR_d.nii.gz
OAS30572_MR_d.nii.gz
OAS31087_MR_d.nii.gz
